# Tutorial for training and running SRGAN for micro-CT image super resolution

This is a tutorial for training and running Super Resolution Generative Adversarial Network (SRGAN) for micro-CT image super resolution.

# Dependencies
Many required packages are installed by default. Here the packages that may require manual installation are listed below

[Python 3.7](https://www.python.org/) (Older version might run into compatibility issues)

[pytorch 1.1.0](https://pytorch.org/) (Older version might run into compatibility issues)

[CUDA 10.1](https://developer.nvidia.com/cuda-zone) (For GPU computing, older version might run into compatibility issues)

[NumPy](https://www.numpy.org/)

[Pillow](https://pillow.readthedocs.io/en/stable/installation.html)

[SciPy](https://www.scipy.org/)

[imageio](https://imageio.github.io/)

[skimage](https://scikit-image.org/docs/dev/api/skimage.html)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import utils
from PIL import Image
import simple_transforms_2d
import dataset_utils
import transforms_3d
import scipy
import misc
import imageio
from skimage.measure import compare_ssim as ssim
from skimage import img_as_float

# Training the model

The commandline arguments for training the model are listed below.


`model_name`: model architecture, default=`SRGAN`, choices=['SRGAN','EDSR'].

`data_dir`: this is the directory of the training dataset, this usually specifies the N value of the LR and HR images, data in this directory should be further split into training and testing set

`train_dataset`: name of the training directory, should be inside `data_dir`, default=`train`
 
`test_dataset`: name of the testing directory, should be inside `data_dir`, default=`test`

`sample_set`: the name of samples train or test, should be inside either `train_dataset` or `test_dataset`, default is to include all samples

`crop_size`: size to crop the HR image during training or testing, `default=128`, must be positive integer.

`num_threads`: number of threads for data loader to use, `default=4`, must be positive integer.

`num_channels`: number of channels in the image, should be 1 for grayscale microCT images, `default=1`

`num_residuals`: number of residual blocks in the network, `default=8`, must be positive integer.

`scale_factor`: scale factor for super resolution, `default=2`, can be 2 or 4

`num_epochs`: the number of epochs to run the training, `default=100`, must be positive integer.

`save_epochs`: save trained model every this epochs, `default=10`, must be positive integer.

`batch_size`: training batch size, `default=8`, must be positive integer.

`test_batch_size`: testing batch size, `default=1`, must be positive integer.

`save_dir`: directory name to save the results to or directory to load model for testing or prediction, `default='Result'`

`lr`: learning rate, `default=0.0001`

`gpu`/`cpu`: flag for whether network will be run on GPU or CPU, default is GPU

`registered`/`downsample`: flag for whether network is trained on registered LR and HR image or downsample HR and HR image, default is using registered LR and HR image pairs(recommended)

`mode`: mode to run the network, default is `train_and_test`, available choices: 
    
    train_and_test: train the network and test on the same dataset
    
    test: test the network on selected dataset
    
    predict: takes 2D raw. image files and generate SR images
    
    predict3d: takes 3D raw. image files and generate 3D SR image
    
`kernel`: size of kernel used for convolutional layers, `default=3`, must be positive integer.

`filter`: number of filters used for convolutional layers, `default=128`, must be positive integer.

`output`: output format for 'predict' mode', default is `png.` file, available choices:
    
    pic: save to png. file
    
    raw: save to raw. file
   
`pretrain`: number of pretrain epoch for generator, `default=0`

`lr_d`: learning rate multiplier for discriminator, `default=0.1` (meaning the learning rate of discriminator is 10 times smaller than the learning rate of generator)

`vgg_factor`: weight of VGG loss in the total loss function, `default=1.0`

`vgg_layer`: VGG layer used to calculate VGG loss, `default=8`

`metric`: The type of score metrics used for testing, `default=sc`, available choices:
    
    sc: structural comparison term in SSIM
    
    ssim: Structural similarity index
    
    psnr: peak signal to noise ratio


`no_grayscale_correction`: flag for if dataset without grayscale histogram matching is used, the default is to use dataset with grayscale correction

# Example for training a model
Note that a test on the training sample set(s) will be automatically performed. 
__Note that the `!` at the start is the IPython magic command necessary to run the code in jupyter notebook. To run this command in command line directly, remove the `!` at the start.__

In [ ]:
! python main.py --model_name SRGAN --data_dir Registered_2D_slices\N6-N12 --sample_set APX134M2 --num_epochs 5 --scale_factor 2 --save_dir Models\test_model




# Example for testing a model with 2D slices
This compares SR image and Bicubically interpolated LR image with HR images and plot them together with metric score. Note that the test is by default done on cropped images. For generating super resolved 2D or 3D image without cropping and saving them to disk, use `predict` or `predict3d` mode

In [ ]:
! python main.py --model_name SRGAN --data_dir Registered_2D_slices\N6-N12 --sample_set APX129M2 --num_epochs 10 --scale_factor 2 --save_dir Models\test_model --mode test


# Example for generating 2D super resolved image
The default is to save super resolved image to `png` file. The images can also be saved to `raw` format

In [ ]:
! python main.py --model_name SRGAN --data_dir Registered_2D_slices\N6-N12 --sample_set APX129M2 --test_dataset single_image --num_epochs 10 --scale_factor 2 --save_dir Models\test_model --mode predict

# Example for generating 3D super resolved image 

Super Resolution of a 3D micro-CT image is done in the following step. __The maximum size of the image is limited by the memory of the system so cropping of the original image may be necessary.__ In this example, all `raw.` files directly under `Registered_2D_slices\N6-N12\predict\APX129M2` directory should be 3D and will be super resolved. The image will be sliced in x, y and z direction and super resolved in each direction. The output 2D images will be saved in the model folder under the directory `Models\test_model\SRGAN\SR-3D\predict\APX129M2` into 3 folders: `x_slice`, `y_slice` and `z_slice`. To reconstruct the 2D slices into 3D volume. Each directional slice is combined and resized into desired output shape with the correct scale factor. Then the average of the three directional reconstruction is taken as the final 3D SR reconstruction. The final 3D SR image will be saved under directory `Models\test_model\SRGAN\SR-3D\predict\APX129M2\recons`

In [ ]:
! python main.py --model_name SRGAN --data_dir Registered_2D_slices\N6-N12 --sample_set APX129M2 --test_dataset predict --num_epochs 10 --scale_factor 2 --save_dir Models\test_model --mode predict3d

In certain cases, the super resolved 3D volume may be too large and run into memory problem during segmentation. One way to overcome this is to crop the whole 3D volume into smaller cubes, run super resolution/segmenation individually and combining them back together. The following is a function for combining smaller 3D volume into one large 3D volume.

In [ ]:
image_dir='Result\\result_comparison\\RCC\\16750SEG'
utils.combine_cropped(image_dir,crop_factor=2,dtype='uint8')

# Addition of New Training/Testing Data

For new training/testing data. The HR/LR image pairs should be registered and aligned center by center and saved in the same folder. The N value should be calculated or estimated for both images. The `register_and_correct` function takes in LR and HR image paths, original N values for LR and HR images, desired N values to be resized to for LR and HR images, and a reference python slice index (starts at 0) for corresponding LR and HR image (will be center to center if the images are aligned properly). At this step, the LR and HR image are sliced along z direction and grayscale histogram matching and expansion/cropping adjustment is made if necessary.

In [2]:
filepath_hr='3D_Rock_Volumes\\GB1\\GB1-scan2_2.05um_16bu_m0.75_1024x1024x1024.raw'
filepath_lr='3D_Rock_Volumes\\GB1\\GB1-scan1_5.03um_16bu_m0.75_413x413x413.raw'
utils.register_and_correct(filepath_lr,filepath_hr,(2.01087,4.934),(2,4),(206,511))

adjust LR images size:5
std:1559.7750568859024
adjust LR images size:4
std:1528.06727914494
adjust LR images size:3
std:1453.6823820118188
adjust LR images size:2
std:1328.599925057804
adjust LR images size:1
std:1169.3141245098855
adjust HR images size:0
std:1013.1593017578125
adjust HR images size:1
std:1043.775634765625
adjust HR images size:2
std:1110.794189453125
adjust HR images size:3
std:1201.235107421875
adjust HR images size:4
std:1292.5040283203125
adjust HR images size:5
std:1369.6279296875
adjust HR images size:6
std:1435.84814453125
adjust HR images size:7
std:1485.035400390625
adjust HR images size:8
std:1521.9056396484375
adjust HR images size:9
std:1547.8697509765625
the adjusted size for HR images is:0


After this step, a new folder named the new N value pair such as `N2-N4` will be created in the same directory. Inside the folder will be 3 folders named `lr`, `hr` and `Grayscale-adjusted-lr`. Inside each folder will be the 2D image slices from the sample. __At this time, the data can be further split into training and testing set manually. It is important that the same split is used for all four folders. Spliting the dataset is optional.__

## The next step is important! (Need to adjust/simplify folder creation, sometimes lr get adjusted)

__The new data is now ready to be moved into the dataset. Move the dataset to__ `N%-N%/train` __and/or__ `N%-N%/test` __folder with corresponding N value pair and whether or not data is split into training and testing set. The data should be in a folder with its sample name. For example, the `lr`, `hr` and `Grayscale-adjusted-lr` folders should be in a folder with the path `N2-N4/train/GB1`. This step must be done manually!__

# Example for caculating average PSNR for a series of images

In [91]:
dir1='Result\\result_comparison\\N2-N4\\CWP060M1validation\\HR\\seg'
dir2='Result\\result_comparison\\N2-N4\\CWP060M1validation\\SRGAN_N2-4_CWP82_96_126_batch16\\seg'

In [92]:
utils.calculate_average_PSNR(dir1,dir2)

average psnr is:10.240693113996425


10.240693113996425

# Example for calculating PSNR, SSIM or Structural comparison between two images

In [5]:
hr_image_path='Result_and_plots\\traditional PSNR and SSIM metrics\\APX\\Expansion-adjusted-hr-312--APX129M2-scan3_1.97um_16bu_m0.75_200x200.png'
sr_image_path='Result_and_plots\\traditional PSNR and SSIM metrics\\APX\\SRGAN_N6-12_APX134M2_vgg1_res8_grayscale.png'
bc_image_path='Result_and_plots\\traditional PSNR and SSIM metrics\\APX\\bicubic-lr-312--APX129M2-scan2_4.04um_16bu_m0.75_100x100.png'

In [6]:
hr_image=Image.open(hr_image_path)
hr_image=np.array(hr_image).astype(float)
hr_image=hr_image/256
sr_image=Image.open(sr_image_path)
sr_image=np.array(sr_image).astype(float)
sr_image=sr_image/256
bc_image=Image.open(bc_image_path)
bc_image=np.array(bc_image).astype(float)
bc_image=bc_image/256

In [8]:
utils.PSNR(sr_image,hr_image)

17.706861638743135

In [10]:
ssim(hr_image,sr_image,data_range=1.0)

0.7377743692768679

In [12]:
misc.structural_similarity(hr_image,bc_image, data_range=1.0)

0.8573471190831768